In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics
import scipy.stats
import os
import re

In [ ]:
df_all = []
# get all filnames beginning with results/geodesic_optimization_results_
filenames = os.listdir('./results/')
print(len(filenames))
filenames = [f for f in filenames if re.match(
    r'geodesic_optimization_results_\d+', f
    )]
filenames = sorted(filenames, key=lambda x: int(re.search(r'\d+', x).group()))
for filename in filenames:
    df = pd.read_csv(f'./results/{filename}', index_col=0)
    df_all.append(df)
df_all = pd.concat(df_all)
df_all.reset_index(inplace=True, drop=True)

In [15]:
import numpy as np
from scipy.stats import t


def corrected_std(differences, n_train, n_test):
    """Corrects standard deviation using Nadeau and Bengio's approach.

    Parameters
    ----------
    differences : ndarray of shape (n_samples,)
        Vector containing the differences in the score metrics of two models.
    n_train : int
        Number of samples in the training set.
    n_test : int
        Number of samples in the testing set.

    Returns
    -------
    corrected_std : float
        Variance-corrected standard deviation of the set of differences.
    """
    # kr = k times r, r times repeated k-fold crossvalidation,
    # kr equals the number of times the model was evaluated
    kr = len(differences)
    corrected_var = np.var(differences, ddof=1) * (1 / kr + n_test / n_train)
    corrected_std = np.sqrt(corrected_var)
    return corrected_std


def compute_corrected_ttest(differences, df, n_train, n_test):
    """Computes right-tailed paired t-test with corrected variance.

    Parameters
    ----------
    differences : array-like of shape (n_samples,)
        Vector containing the differences in the score metrics of two models.
    df : int
        Degrees of freedom.
    n_train : int
        Number of samples in the training set.
    n_test : int
        Number of samples in the testing set.

    Returns
    -------
    t_stat : float
        Variance-corrected t-statistic.
    p_val : float
        Variance-corrected p-value.
    """
    mean = np.mean(differences)
    std = corrected_std(differences, n_train, n_test)
    t_stat = mean / std
    return t_stat


def get_p_val(t_stat, df=100):
    p_val = t.sf(np.abs(t_stat), df)  # right-tailed t-test
    return p_val


In [21]:
source_sizes = {1: 739, 2: 389, 3: 294, 4: 592, 5: 1093}
contrast_pairs = [
    ('dummy', 'baseline_no_recenter'),
    ('dummy', 'baseline_recenter'),
    ('baseline_green', 'baseline_fit_intercept'),
    ('baseline_green', 'geodesic_optim'),
    ('baseline_no_recenter', 'baseline_fit_intercept'),
    ('baseline_fit_intercept', 'geodesic_optim')
]

In [22]:
p_vals = list()
for ref, pro in contrast_pairs:
    for source in df_all.sites_source_index.unique():
    # for source in [1]:
        source = int(source)
    #     print(source, target)
        df_ref = df_all.query(
            f"method == '{ref}' & sites_source_index == {source}"
        ).reset_index(drop=True)
        df_pro = df_all.query(
            f"method == '{pro}' & sites_source_index == {source}"
        ).reset_index(drop=True)

        assert np.all(df_ref.index == df_pro.index)
        assert np.all(df_ref.y_true == df_pro.y_true)
        differences = list()
        for gg, index in df_ref.groupby('split_index').groups.items():
            df_sub1 = df_ref.iloc[index]
            df_sub2 = df_pro.iloc[index]
            sp_ref = scipy.stats.spearmanr(df_sub1['y_true'], df_sub1['y_pred']).statistic
            sp_pro = scipy.stats.spearmanr(df_sub2['y_true'], df_sub2['y_pred']).statistic
            mae_ref = sklearn.metrics.mean_absolute_error(df_sub1['y_true'], df_sub1['y_pred'])
            mae_pro = sklearn.metrics.mean_absolute_error(df_sub2['y_true'], df_sub2['y_pred'])
            r2_ref = sklearn.metrics.r2_score(df_sub1['y_true'], df_sub1['y_pred'])
            r2_pro = sklearn.metrics.r2_score(df_sub2['y_true'], df_sub2['y_pred'])

            differences.append(
                dict(spearman=sp_pro - sp_ref,
                     mae=mae_pro - mae_ref,
                     r2=r2_pro - r2_ref)
            )
        differences = pd.DataFrame(differences)
        n_train = source_sizes[source]
        n_test = int(len(df_sub1['y_pred']) // 2)
        df_p = differences.apply(lambda x: get_p_val(compute_corrected_ttest(differences=x, df=100, n_train=n_train, n_test=n_test)))
        df_p['contr'] = f'{pro}-{ref}'
        df_p['source'] = source
        p_vals.append(df_p)


In [23]:
df_pval = pd.concat(p_vals, axis=1).T

In [ ]:
df_pval

In [20]:
df_pval.to_csv('./results/p_values.csv')